In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install ultralytics
!pip install shapely

In [2]:
from shapely.geometry import box

def iou_val(box1, box2):
  if box1.intersects(box2):
    iou = (box1.intersection(box2).area) / (box1.union(box2).area)
    return iou
  return 0


In [8]:
# Identify Leader-Follower Pairs (Blue=Leader, Red=Follower) for:
# cars moving right:
def dir_right(box1, boxes, frame, count, leader_follower_data):
  # Logic to detect vehicles in the range of vehicle1 (range = width/2 and length as the dimensions):
  sbox1 = box(box1_right, (box1_top-(box1_h)*0.5), (box1_right+box1_w), box1_top)
  sbox2 = box(box1_right, box1_top, (box1_right+box1_w), box1_bot)
  sbox3 = box(box1_right, box1_bot, (box1_right+box1_w), (box1_bot+(box1_h)*0.5))
  sbox4 = box(box1_left, (box1_top-(box1_h)*0.5), box1_right, box1_top)
  sbox5 = box(box1_left, box1_bot, box1_right, (box1_bot+(box1_h)*0.5))
  sbox6 = box((box1_left-box1_w), (box1_top-(box1_h)*0.5), box1_left, box1_top)
  sbox7 = box((box1_left-box1_w), box1_top, box1_left, box1_bot)
  sbox8 = box((box1_left-box1_w), box1_bot, box1_left, (box1_bot+(box1_h)*0.5))
  sbox_arr = [sbox1,sbox2,sbox3,sbox4,sbox5,sbox6,sbox7,sbox8]

  for box2 in boxes:
    if box1==box2:
      continue
    else:
      box2_w = box2.xywh[0][2]
      box2_h = box2.xywh[0][3]

      box2_left, box2_top, box2_right, box2_bot = box2.xyxy[0]

      box2_cx = (box2_left+box2_right)*0.5
      box2_cy = (box2_top+box2_bot)*0.5

      boxx = box(box2_left, box2_top, box2_right, box2_bot)
      max_iou = max(iou_val(sbox, boxx) for sbox in sbox_arr)
      if max_iou==0:
        continue
      elif iou_val(sbox1, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, int(box2.id), 0, 0, 0, 0, 0, 0, 0]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
      elif iou_val(sbox2, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, 0, int(box2.id), 0, 0, 0, 0, 0, 0]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
      elif iou_val(sbox3, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, 0, 0, int(box2.id), 0, 0, 0, 0, 0]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
      elif iou_val(sbox4, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, 0, 0, 0, int(box2.id), 0, 0, 0, 0]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
      elif iou_val(sbox5, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, 0, 0, 0, 0, int(box2.id), 0, 0, 0]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
      elif iou_val(sbox6, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, 0, 0, 0, 0, 0, int(box2.id), 0, 0]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
        # Logic to identify Leader-Follower Pairs:
        if (box1_top<box2_top<box1_bot or box1_top<box2_bot<box1_bot) or box1_top<box2_cy<box1_bot: # Checking width zone (height of leader car)
          if box1_left-box1_w<box2_right<box1_left: # Checking length zone (width of leader car)
            LT = 0.5*(box1_h) + box2_h # Calculating Lateral Threshold
            LG = abs(box1_cy - box2_cy) # Calculating Lateral Gap
            if LG<LT:
              cv2.rectangle(frame, (int(box1_left), int(box1_top)), (int(box1_right), int(box1_bot)), (255, 0, 0), 3)
              cv2.rectangle(frame, (int(box2_left), int(box2_top)), (int(box2_right), int(box2_bot)), (0, 0, 255), 3)

              df1['Follower'] = [int(box2.id)]
      elif iou_val(sbox7, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, 0, 0, 0, 0, 0, 0, int(box2.id), 0]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
        # Logic to identify Leader-Follower Pairs:
        if (box1_top<box2_top<box1_bot or box1_top<box2_bot<box1_bot) or box1_top<box2_cy<box1_bot: # Checking width zone (height of leader car)
          if box1_left-box1_w<box2_right<box1_left: # Checking length zone (width of leader car)
            LT = 0.5*(box1_h) + box2_h # Calculating Lateral Threshold
            LG = abs(box1_cy - box2_cy) # Calculating Lateral Gap
            if LG<LT:
              cv2.rectangle(frame, (int(box1_left), int(box1_top)), (int(box1_right), int(box1_bot)), (255, 0, 0), 3)
              cv2.rectangle(frame, (int(box2_left), int(box2_top)), (int(box2_right), int(box2_bot)), (0, 0, 255), 3)

              df1['Follower'] = [int(box2.id)]
      elif iou_val(sbox8, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, 0, 0, 0, 0, 0, 0, 0, int(box2.id)]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
        # Logic to identify Leader-Follower Pairs:
        if (box1_top<box2_top<box1_bot or box1_top<box2_bot<box1_bot) or box1_top<box2_cy<box1_bot: # Checking width zone (height of leader car)
          if box1_left-box1_w<box2_right<box1_left: # Checking length zone (width of leader car)
            LT = 0.5*(box1_h) + box2_h # Calculating Lateral Threshold
            LG = abs(box1_cy - box2_cy) # Calculating Lateral Gap
            if LG<LT:
              cv2.rectangle(frame, (int(box1_left), int(box1_top)), (int(box1_right), int(box1_bot)), (255, 0, 0), 3)
              cv2.rectangle(frame, (int(box2_left), int(box2_top)), (int(box2_right), int(box2_bot)), (0, 0, 255), 3)

              df1['Follower'] = [int(box2.id)]

      leader_follower_data = pd.concat([leader_follower_data, df1], ignore_index=True)
  return leader_follower_data

In [9]:
# cars moving left:
def dir_left(box1, boxes, frame, count, leader_follower_data):
  # Logic to detect vehicles in the range of vehicle1 (range = width/2 and length as the dimensions):
  sbox1 = box((box1_left-box1_w), box1_bot, box1_left, (box1_bot+(box1_h)*0.5))
  sbox2 = box((box1_left-box1_w), box1_top, box1_left, box1_bot)
  sbox3 = box((box1_left-box1_w), (box1_top-(box1_h)*0.5), box1_left, box1_top)
  sbox4 = box(box1_left, box1_bot, box1_right, (box1_bot+(box1_h)*0.5))
  sbox5 = box(box1_left, (box1_top-(box1_h)*0.5), box1_right, box1_top)
  sbox6 = box(box1_right, box1_bot, (box1_right+box1_w),( box1_bot+(box1_h)*0.5))
  sbox7 = box(box1_right, box1_top, (box1_right+box1_w), box1_bot)
  sbox8 = box(box1_right, (box1_top-(box1_h)*0.5), (box1_right+box1_w), box1_top)
  sbox_arr = [sbox1,sbox2,sbox3,sbox4,sbox5,sbox6,sbox7,sbox8]

  for box2 in boxes:
    if box1==box2:
      continue
    else:
      box2_w = box2.xywh[0][2]
      box2_h = box2.xywh[0][3]

      box2_left, box2_top, box2_right, box2_bot = box2.xyxy[0]

      box2_cx = (box2_left+box2_right)*0.5
      box2_cy = (box2_top+box2_bot)*0.5

      boxx = box(box2_left, box2_top, box2_right, box2_bot)
      max_iou = max(iou_val(sbox, boxx) for sbox in sbox_arr)
      if max_iou==0:
        continue
      elif iou_val(sbox1, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, int(box2.id), 0, 0, 0, 0, 0, 0, 0]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
      elif iou_val(sbox2, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, 0, int(box2.id), 0, 0, 0, 0, 0, 0]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
      elif iou_val(sbox3, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, 0, 0, int(box2.id), 0, 0, 0, 0, 0]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
      elif iou_val(sbox4, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, 0, 0, 0, int(box2.id), 0, 0, 0, 0]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
      elif iou_val(sbox5, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, 0, 0, 0, 0, int(box2.id), 0, 0, 0]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
      elif iou_val(sbox6, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, 0, 0, 0, 0, 0, int(box2.id), 0, 0]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
        if (box1_top<box2_top<box1_bot or box1_top<box2_bot<box1_bot) or box1_top<box2_cy<box1_bot: # Checking width zone (height of leader car)
          if box1_right+box1_w>box2_left>box1_right: # Checking length zone (width of leader car)
            LT = 0.5*(box1_h) + box2_h # Calculating Lateral Threshold
            LG = abs(box1_cy - box2_cy) # Calculating Lateral Gap
            if LG<LT:
              cv2.rectangle(frame, (int(box1_left), int(box1_top)), (int(box1_right), int(box1_bot)), (255, 0, 0), 3)
              cv2.rectangle(frame, (int(box2_left), int(box2_top)), (int(box2_right), int(box2_bot)), (0, 0, 255), 3)

              df1['Follower'] = [int(box2.id)]
      elif iou_val(sbox7, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, 0, 0, 0, 0, 0, 0, int(box2.id), 0]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
        if (box1_top<box2_top<box1_bot or box1_top<box2_bot<box1_bot) or box1_top<box2_cy<box1_bot: # Checking width zone (height of leader car)
          if box1_right+box1_w>box2_left>box1_right: # Checking length zone (width of leader car)
            LT = 0.5*(box1_h) + box2_h # Calculating Lateral Threshold
            LG = abs(box1_cy - box2_cy) # Calculating Lateral Gap
            if LG<LT:
              cv2.rectangle(frame, (int(box1_left), int(box1_top)), (int(box1_right), int(box1_bot)), (255, 0, 0), 3)
              cv2.rectangle(frame, (int(box2_left), int(box2_top)), (int(box2_right), int(box2_bot)), (0, 0, 255), 3)

              df1['Follower'] = [int(box2.id)]
      elif iou_val(sbox8, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, 0, 0, 0, 0, 0, 0, 0, int(box2.id)]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
        if (box1_top<box2_top<box1_bot or box1_top<box2_bot<box1_bot) or box1_top<box2_cy<box1_bot: # Checking width zone (height of leader car)
          if box1_right+box1_w>box2_left>box1_right: # Checking length zone (width of leader car)
            LT = 0.5*(box1_h) + box2_h # Calculating Lateral Threshold
            LG = abs(box1_cy - box2_cy) # Calculating Lateral Gap
            if LG<LT:
              cv2.rectangle(frame, (int(box1_left), int(box1_top)), (int(box1_right), int(box1_bot)), (255, 0, 0), 3)
              cv2.rectangle(frame, (int(box2_left), int(box2_top)), (int(box2_right), int(box2_bot)), (0, 0, 255), 3)

              df1['Follower'] = [int(box2.id)]

      leader_follower_data = pd.concat([leader_follower_data, df1], ignore_index=True)
  return leader_follower_data


In [10]:
# cars moving up:
def dir_up(box1, boxes, frame, count, leader_follower_data):
  # Logic to detect vehicles in the range of vehicle1 (range = width/2 and length as the dimensions):
  sbox1 = box((box1_left-box1_w), (box1_top-(box1_h)*0.5), box1_left, box1_top)
  sbox2 = box(box1_left, (box1_top-(box1_h)*0.5), box1_right, box1_top)
  sbox3 = box(box1_right, (box1_top-(box1_h)*0.5), (box1_right+box1_w), box1_top)
  sbox4 = box((box1_left-box1_w), box1_top, box1_left, box1_bot)
  sbox5 = box(box1_right, box1_top, (box1_right+box1_w), box1_bot)
  sbox6 = box((box1_left-box1_w), box1_bot, box1_left, (box1_bot+(box1_h)*0.5))
  sbox7 = box(box1_left, box1_bot, box1_right, (box1_bot+(box1_h)*0.5))
  sbox8 = box(box1_right, box1_bot, (box1_right+box1_w), (box1_bot+(box1_h)*0.5))
  sbox_arr = [sbox1,sbox2,sbox3,sbox4,sbox5,sbox6,sbox7,sbox8]

  for box2 in boxes:
    if box1==box2:
      continue
    else:
      box2_w = box2.xywh[0][2]
      box2_h = box2.xywh[0][3]

      box2_left, box2_top, box2_right, box2_bot = box2.xyxy[0]

      box2_cx = (box2_left+box2_right)*0.5
      box2_cy = (box2_top+box2_bot)*0.5

      boxx = box(box2_left, box2_top, box2_right, box2_bot)
      max_iou = max(iou_val(sbox, boxx) for sbox in sbox_arr)
      if max_iou==0:
        continue
      elif iou_val(sbox1, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, int(box2.id), 0, 0, 0, 0, 0, 0, 0]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
      elif iou_val(sbox2, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, 0, int(box2.id), 0, 0, 0, 0, 0, 0]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
      elif iou_val(sbox3, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, 0, 0, int(box2.id), 0, 0, 0, 0, 0]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
      elif iou_val(sbox4, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, 0, 0, 0, int(box2.id), 0, 0, 0, 0]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
      elif iou_val(sbox5, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, 0, 0, 0, 0, int(box2.id), 0, 0, 0]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
      elif iou_val(sbox6, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, 0, 0, 0, 0, 0, int(box2.id), 0, 0]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
        if (box1_left<box2_left<box1_right or box1_left<box2_right<box1_right) or box1_left<box2_cx<box1_right: # Checking width zone (width of leader car)
          if box1_bot+box1_h>box2_top>box1_bot: # Checking length zone (height of leader car)
            LT = 0.5*(box1_w) + box2_w # Calculating Lateral Threshold
            LG = abs(box1_cx - box2_cx) # Calculating Lateral Gap
            if LG<LT:
              cv2.rectangle(frame, (int(box1_left), int(box1_top)), (int(box1_right), int(box1_bot)), (255, 0, 0), 3)
              cv2.rectangle(frame, (int(box2_left), int(box2_top)), (int(box2_right), int(box2_bot)), (0, 0, 255), 3)

              df1['Follower'] =[int(box2.id)]

      elif iou_val(sbox7, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, 0, 0, 0, 0, 0, 0, int(box2.id), 0]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
        if (box1_left<box2_left<box1_right or box1_left<box2_right<box1_right) or box1_left<box2_cx<box1_right: # Checking width zone (width of leader car)
          if box1_bot+box1_h>box2_top>box1_bot: # Checking length zone (height of leader car)
            LT = 0.5*(box1_w) + box2_w # Calculating Lateral Threshold
            LG = abs(box1_cx - box2_cx) # Calculating Lateral Gap
            if LG<LT:
              cv2.rectangle(frame, (int(box1_left), int(box1_top)), (int(box1_right), int(box1_bot)), (255, 0, 0), 3)
              cv2.rectangle(frame, (int(box2_left), int(box2_top)), (int(box2_right), int(box2_bot)), (0, 0, 255), 3)

              df1['Follower'] =[int(box2.id)]
      elif iou_val(sbox8, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, 0, 0, 0, 0, 0, 0, 0, int(box2.id)]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
        if (box1_left<box2_left<box1_right or box1_left<box2_right<box1_right) or box1_left<box2_cx<box1_right: # Checking width zone (width of leader car)
          if box1_bot+box1_h>box2_top>box1_bot: # Checking length zone (height of leader car)
            LT = 0.5*(box1_w) + box2_w # Calculating Lateral Threshold
            LG = abs(box1_cx - box2_cx) # Calculating Lateral Gap
            if LG<LT:
              cv2.rectangle(frame, (int(box1_left), int(box1_top)), (int(box1_right), int(box1_bot)), (255, 0, 0), 3)
              cv2.rectangle(frame, (int(box2_left), int(box2_top)), (int(box2_right), int(box2_bot)), (0, 0, 255), 3)

              df1['Follower'] =[int(box2.id)]

      leader_follower_data = pd.concat([leader_follower_data, df1], ignore_index=True)
  return leader_follower_data


In [6]:
# cars moving down:
def dir_down(box1, boxes, frame, count, leader_follower_data):
  # Logic to detect vehicles in the range of vehicle1 (range = width/2 and length as the dimensions):
  sbox1 = box(box1_right, box1_bot, (box1_right+box1_w), (box1_bot+(box1_h)*0.5))
  sbox2 = box(box1_left, box1_bot, box1_right, (box1_bot+(box1_h)*0.5))
  sbox3 = box((box1_left-box1_w), box1_bot, box1_left, (box1_bot+(box1_h)*0.5))
  sbox4 = box(box1_right, box1_top, (box1_right+box1_w), box1_bot)
  sbox5 = box((box1_left-box1_w), box1_top, box1_left, box1_bot)
  sbox6 = box(box1_right, (box1_top-(box1_h)*0.5), (box1_right+box1_w), box1_top)
  sbox7 = box(box1_left, (box1_top-(box1_h)*0.5), box1_right, box1_top)
  sbox8 = box((box1_left-box1_w), (box1_top-(box1_h)*0.5), box1_left, box1_top)
  sbox_arr = [sbox1,sbox2,sbox3,sbox4,sbox5,sbox6,sbox7,sbox8]

  for box2 in boxes:
    if box1==box2:
      continue
    else:
      box2_w = box2.xywh[0][2]
      box2_h = box2.xywh[0][3]

      box2_left, box2_top, box2_right, box2_bot = box2.xyxy[0]

      box2_cx = (box2_left+box2_right)*0.5
      box2_cy = (box2_top+box2_bot)*0.5

      boxx = box(box2_left, box2_top, box2_right, box2_bot)
      max_iou = max(iou_val(sbox, boxx) for sbox in sbox_arr)
      if max_iou==0:
        continue
      elif iou_val(sbox1, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, int(box2.id), 0, 0, 0, 0, 0, 0, 0]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
      elif iou_val(sbox2, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, 0, int(box2.id), 0, 0, 0, 0, 0, 0]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
      elif iou_val(sbox3, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, 0, 0, int(box2.id), 0, 0, 0, 0, 0]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
      elif iou_val(sbox4, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, 0, 0, 0, int(box2.id), 0, 0, 0, 0]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
      elif iou_val(sbox5, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, 0, 0, 0, 0, int(box2.id), 0, 0, 0]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
      elif iou_val(sbox6, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, 0, 0, 0, 0, 0, int(box2.id), 0, 0]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
        if (box1_left<box2_left<box1_right or box1_left<box2_right<box1_right) or box1_left<box2_cx<box1_right: # Checking width zone (width of leader car)
          if box1_top-box1_h<box2_bot<box1_top: # Checking length zone (height of leader car)
            LT = 0.5*(box1_w) + box2_w # Calculating Lateral Threshold
            LG = abs(box1_cx - box2_cx) # Calculating Lateral Gap
            if LG<LT:
              cv2.rectangle(frame, (int(box1_left), int(box1_top)), (int(box1_right), int(box1_bot)), (255, 0, 0), 3)
              cv2.rectangle(frame, (int(box2_left), int(box2_top)), (int(box2_right), int(box2_bot)), (0, 0, 255), 3)

              df1['Follower'] = [int(box2.id)]
      elif iou_val(sbox7, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, 0, 0, 0, 0, 0, 0, int(box2.id), 0]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
        if (box1_left<box2_left<box1_right or box1_left<box2_right<box1_right) or box1_left<box2_cx<box1_right: # Checking width zone (width of leader car)
          if box1_top-box1_h<box2_bot<box1_top: # Checking length zone (height of leader car)
            LT = 0.5*(box1_w) + box2_w # Calculating Lateral Threshold
            LG = abs(box1_cx - box2_cx) # Calculating Lateral Gap
            if LG<LT:
              cv2.rectangle(frame, (int(box1_left), int(box1_top)), (int(box1_right), int(box1_bot)), (255, 0, 0), 3)
              cv2.rectangle(frame, (int(box2_left), int(box2_top)), (int(box2_right), int(box2_bot)), (0, 0, 255), 3)

              df1['Follower'] = [int(box2.id)]
      elif iou_val(sbox8, boxx)==max_iou:
        data1 = [[count, int(box1.id), 0, 0, 0, 0, 0, 0, 0, 0, int(box2.id)]]
        df1 = pd.DataFrame(data1, columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])
        if (box1_left<box2_left<box1_right or box1_left<box2_right<box1_right) or box1_left<box2_cx<box1_right: # Checking width zone (width of leader car)
          if box1_top-box1_h<box2_bot<box1_top: # Checking length zone (height of leader car)
            LT = 0.5*(box1_w) + box2_w # Calculating Lateral Threshold
            LG = abs(box1_cx - box2_cx) # Calculating Lateral Gap
            if LG<LT:
              cv2.rectangle(frame, (int(box1_left), int(box1_top)), (int(box1_right), int(box1_bot)), (255, 0, 0), 3)
              cv2.rectangle(frame, (int(box2_left), int(box2_top)), (int(box2_right), int(box2_bot)), (0, 0, 255), 3)

              df1['Follower'] = [int(box2.id)]

      leader_follower_data = pd.concat([leader_follower_data, df1], ignore_index=True)
  return leader_follower_data


In [13]:
import cv2
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
import pandas as pd
import numpy as np
import torch

# Load the YOLOv8 model
model = YOLO('/content/drive/MyDrive/copy_best.pt')

# Open the video file
video_path = "/content/drive/MyDrive/WictroniX/test_input.mp4"
cap = cv2.VideoCapture(video_path)

# Get input video properties (e.g., frame width, height, and frame rate)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create a VideoWriter object to write the output video
output_video_path = '/content/drive/MyDrive/WictroniX/test_output.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Change the codec based on the output format
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

prev_pos={}
current_pos={}
count=0
direction="Stationary"

# Create DataFrame to store leader-follower data
leader_follower_data = pd.DataFrame(columns=['Frame', 'Leader', 'Follower', 'Top Left', 'Top Middle', 'Top Right', 'Middle Left', 'Middle Right', 'Bottom Left', 'Bottom Middle', 'Bottom Right'])

# Loop through the video frames
while cap.isOpened():
  # Read a frame from the video
  success, frame = cap.read()

  if success:
    count += 1  # i.e. at 1 frame, this advances one second
    cap.set(cv2.CAP_PROP_POS_FRAMES, count)
    # Run YOLOv8 inference on the frame
    results = model.track(frame, persist=True)
    boxes = results[0].boxes
    ids = boxes.id

    # Store the current detections(bboxes) with their respective ids in the current_pos list
    current_pos={x:y for x,y in zip(ids.tolist(), boxes.xyxy.tolist())}

    for box1, id in zip(boxes, ids.tolist()):
      box1_w = box1.xywh[0][2]
      box1_h = box1.xywh[0][3]

      box1_left, box1_top, box1_right, box1_bot = box1.xyxy[0]

      box1_cx = (box1_left+box1_right)*0.5
      box1_cy = (box1_top+box1_bot)*0.5

      # Drawing bounding boxes with their ID on the current frame
      cv2.rectangle(frame, (int(box1_left), int(box1_top)), (int(box1_right), int(box1_bot)), (0, 255, 0), 1)
      cv2.putText(
        frame,
        f"Id {id}",
        (int(box1_left), int(box1_top)),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 0, 255),
        2,
      )
      if ((id in prev_pos.keys()) and (id in current_pos.keys())):
        x_difference = current_pos[id][0] - prev_pos[id][0]
        y_difference = current_pos[id][1] - prev_pos[id][1]

        # Logic to find the direction:
        # if width>height, car moving horizontally
        if (current_pos[id][2]-current_pos[id][0]) > (current_pos[id][3]-current_pos[id][1]):
          if abs(x_difference) > abs(y_difference):
            if x_difference > 0:
              direction = "Right"
            elif x_difference < 0:
              direction = "Left"
            else:
              direction="Stationary"
          else:
            direction="Stationary"

        # if height>width, car moving vertically
        else:
          if abs(x_difference) < abs(y_difference):
            if y_difference > 0:
              direction = "Down"
            elif y_difference < 0:
              direction = "Up"
            else:
              direction="Stationary"
          else:
              direction="Stationary"
      else:
        direction="Stationary"

      # Writing the direction above the bboxes
      x, y = current_pos[id][0:2]
      cv2.putText(frame, direction, (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

      # Comparing every other bbox with the current bbox to find Leader-Follower Pairs
      if direction=="Right":
        leader_follower_data = dir_right(box1, boxes, frame, count, leader_follower_data)

      elif direction=="Left":
        leader_follower_data = dir_left(box1, boxes, frame, count, leader_follower_data)

      elif direction=="Up":
        leader_follower_data = dir_up(box1, boxes, frame, count, leader_follower_data)

      elif direction=="Down":
        leader_follower_data = dir_down(box1, boxes, frame, count, leader_follower_data)

    prev_pos = current_pos
    if count>=int(cap.get(cv2.CAP_PROP_FRAME_COUNT)):
      break

    # Write the processed frame to the output video
    out.write(frame)

# Save the data to csv
leader_follower_data.to_csv('/content/drive/MyDrive/WictroniX/leader_follower_data.csv', index=False)

cap.release()
out.release()

Streaming output truncated to the last 5000 lines.
0: 384x640 1 Bus, 31 Carss, 1 LCV, 1 MAV, 5 Rickshaws, 1 Truck, 821.1ms
Speed: 10.6ms preprocess, 821.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Bus, 30 Carss, 1 LCV, 5 Rickshaws, 2 Trucks, 855.8ms
Speed: 5.2ms preprocess, 855.8ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Bus, 30 Carss, 2 LCVs, 5 Rickshaws, 1 Truck, 986.4ms
Speed: 4.5ms preprocess, 986.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Bus, 28 Carss, 4 LCVs, 5 Rickshaws, 1 Truck, 1375.5ms
Speed: 4.8ms preprocess, 1375.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Bus, 28 Carss, 2 LCVs, 5 Rickshaws, 2 Trucks, 831.2ms
Speed: 3.6ms preprocess, 831.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Bus, 27 Carss, 2 LCVs, 5 Rickshaws, 3 Trucks, 815.8ms
Speed: 8.3ms preprocess, 815.8ms inference, 1.0